### 1. Analisis I/O Management (dengan Bukti Output)

1.1 Mekanisme I/O yang Digunakan Sistem

Sistem Go-Food multithreading menggunakan I/O berbasis interrupt melalui stdout.
Setiap thread menulis output menggunakan printf(), yang kemudian ditangani oleh OS kernel.

In [ ]:
[Payment] Order 0 dibayar (Rp 49719) - Total Revenue: Rp 49719
[Kitchen 1] Masak order 0
[Kitchen 2] WARNING: Order 1 belum dibayar tapi diproses!

Potongan ini menunjukkan:

- Banyak thread melakukan operasi I/O secara bersamaan

- Penulisan output tidak dikontrol

- OS menangani I/O melalui buffering dan interrupt

Polling tidak digunakan, karena tidak ada loop pengecekan status I/O secara manual.
DMA tidak digunakan, karena data hanya berupa teks berukuran kecil.

1.2 Potensi Bottleneck I/O

Bottleneck utama pada sistem ini adalah stdout sebagai shared resource.

In [ ]:
[Kitchen 2] WARNING: Order 7 belum dibayar tapi diproses!
[Kitchen 4] WARNING: Order 8 belum dibayar tapi diproses!
[Print] Cetak struk untuk order yang selesai... (total=4)

Analisis:

- Banyak thread menulis ke stdout secara bersamaan

- Terjadi interleaving output

- Urutan log tidak mencerminkan alur bisnis yang benar

Selain itu, program berjalan terus-menerus dan harus dihentikan manual: Program harus dihentikan menggunakan Ctrl+C

Ini menunjukkan adanya blocking I/O dan tidak adanya mekanisme termination otomatis.

1.3 Cara OS Nyata Mengatasi Masalah I/O

- OS nyata mengatasi masalah ini dengan:

- Buffering untuk mengurangi overhead I/O

- Mutex untuk melindungi shared resource

- Asynchronous I/O agar thread tidak terblokir

In [ ]:
[Payment] Order 0 dibayar
[Kitchen] Masak order 0
[Payment] Order 1 dibayar
[Kitchen] Masak order 1


### 2. Analisis Security & Protection (dengan Bukti Output)

#### 2.1 Threat Model dan Potential Attacks
a. Race Condition sebagai Ancaman Keamanan

Race condition merupakan ancaman utama karena dapat menyebabkan data inconsistency.

In [ ]:
[Kitchen 3] WARNING: Order 2 belum dibayar tapi diproses!
[Kitchen 4] WARNING: Order 3 belum dibayar tapi diproses!


Dampak keamanan:

- Pelanggaran logika bisnis

- Transaksi tidak valid

- Potensi fraud pada sistem nyata

b. Denial of Service (DoS)

Program berjalan tanpa batas dan harus dihentikan manual.

In [ ]:
[Print] Cetak struk...
[Stock] Update stok... (current=71)
[Print] Cetak struk...

Tanpa mekanisme stop:

- Thread akan terus berjalan

- CPU dan I/O resource bisa habis

- Ini menyerupai DoS internal

#### 2.2 Access Matrix (dengan Justifikasi)

| Object       | Payment    | Kitchen    | Stock      | Print |
| ------------ | ---------- | ---------- | ---------- | ----- |
| Order Queue  | Read       | Read       | No         | No    |
| Payment Data | Read/Write | No         | No         | No    |
| Stock Data   | No         | Read/Write | Read/Write | No    |
| stdout       | Write      | Write      | Write      | Write |


In [ ]:
[Payment] ...
[Kitchen] ...
[Stock] ...
[Print] ...

Karena semua thread memiliki akses tulis ke stdout, mutex diperlukan sebagai mekanisme proteksi.

#### 2.3 Mekanisme Proteksi

Proteksi dilakukan menggunakan mutex, yang:

- Mencegah race condition

- Menjamin atomic access ke resource kritis

- Menyerupai mekanisme lock pada OS nyata

In [ ]:
[Payment] Order dibayar
[Kitchen] Masak order
[Stock] Update stok


Tidak ada lagi warning atau urutan logika yang salah.

### Kesimpulan

Eksperimen menunjukkan bahwa tanpa sinkronisasi, sistem multithreading rentan terhadap race condition dan masalah I/O. Dengan penerapan mutex, OS dapat mengelola I/O dan akses resource secara aman dan konsisten, menyerupai mekanisme yang digunakan pada sistem operasi nyata.